<a href="https://colab.research.google.com/github/JoanChege/CHATBOT1/blob/main/Chatbottrial8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. IMPORT EVERYTHING
2.   SET JSON FILE AS PANDAS DATAFRAME
3.   PREPROCESS THE DATA
4.   SPLIT DATA INTO TRAINING AND TESTING DATA





In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import SGD
#from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import re
import nltk
import json
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
#import the dataset
with open('/content/dataset3.json') as f:
    intents = json.load(f)




# Preprocess Data-remove stopwords
stop_words = set(stopwords.words('english'))

data = []
labels = []

#perform tokenization, filtering, lemmatizatiopn
if intents is not None:
  for intent in intents['intents']:
    for pattern in intent['patterns']:
        pattern = pattern.strip() # Remove leading/trailing whitespace
        tokenized_pattern = nltk.word_tokenize(pattern)
        filtered_pattern = [word.lower() for word in tokenized_pattern if word.isalnum() and not word in stop_words]
        lemmatizer = WordNetLemmatizer()
        lemmatized_pattern = lemmatized_pattern = [lemmatizer.lemmatize(word) for word in filtered_pattern]
        data.append(' '.join(lemmatized_pattern))
        labels.append(intent['tag'])
#The data list contains all the patterns, and the labels list contains all the corresponding intent tags.
# Create a Pandas DataFrame
df = pd.DataFrame({'Patterns': data, 'Tags': labels})

# Encode the Labels by converting tags into numerical form
encoder = LabelEncoder()
df['Tags'] = encoder.fit_transform(df['Tags'])
df['Tags'] = to_categorical(df['Tags'])

The encoding of the labels: The output of this code will be a DataFrame where the 'Tags' column has been converted into one-hot encoded binary values. This is useful for feeding the data into a neural network for text classification, where each class of text can be represented as a one-hot encoded vector.




In [12]:
df

,Patterns,Tags
0,hi,0.0
1,hey,0.0
2,is anyone,0.0
3,hi,0.0
4,hello,0.0
...,...,...
5986,i think absolutely i need i think i realized t...,0.0
5987,what i supposed i ca get head i depressed some...,0.0
5988,med counseling sorted,0.0
5989,deleted,0.0


In [3]:
import collections
word_counts = collections.Counter(data)
num_words = len(word_counts)

print(f"The number of unique words in the vocabulary is {num_words}")

The number of unique words in the vocabulary is 5789


PERFORM FEATURE EXTRACTION USING TDIF

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
#The CountVectorizer converts the text data into numerical vectors by counting the occurrence of each word in the text
vectorizer = CountVectorizer()
vectorizer.fit(data)

CountVectorizer()

In [5]:
X = vectorizer.transform(data) # transform text to feature vectors

In [15]:
y = df['Tags']

In [16]:
train_ratio = 0.8 #80%
val_ratio = 0.1 #10%
test_ratio = 0.1 #10%

In [17]:
from sklearn.utils import shuffle
X, y = shuffle(X, y)

In [19]:
total = X.shape[0]

train_size = int(train_ratio * total)
val_size = int(val_ratio * total)
test_size = int(test_ratio * total)

In [20]:
# Encode labels as unique intergers
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(labels)
#y_train = np.array(labels)


In [22]:
print(X.shape[0])
print(len(y))

5991
5991


In [24]:
# Split into train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Split train into train/validation
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

In [ ]:
print(X_train)

In [26]:
print(X_train.shape[0])
print(len(y_train))

3833
3833


In [ ]:
print(X_test)

In [ ]:
print(y_train)

In [ ]:
print(y_test)

In [28]:
num_classes = len(le.classes_)
y_train = np.reshape(y_train, (-1,1))

In [29]:
model = Sequential()
model.add(Embedding(len(vectorizer.vocabulary_), 100, input_length=X_train.shape[1]))
model.add(Bidirectional(LSTM(64, return_sequences=False)))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [30]:
print(y_train.shape)
print(model.predict(X_train).shape)

(3833, 1)
120/120 [==============================] - 32s 226ms/step
(3833, 318)


In [31]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 10412, 100)        1041200   
                                                                 
 bidirectional (Bidirection  (None, 128)               84480     
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 318)               20670     
                                                        

In [32]:
y_train = y_train.reshape(-1, 1)

In [33]:
num_classes = len(np.unique(y_train))

In [34]:
y_train = np.resize(y_train, (num_classes,))

In [35]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer = SGD(learning_rate=0.01,
                            momentum=0.9, #help to improve the training speed and model stability
                            nesterov=True), metrics=['accuracy'])

In [36]:
num_classes = model.layers[-1].units

In [37]:
y_train = np.resize(y_train, (X_train.shape[0],))

In [38]:
print(X_train.shape)
print(y_train.shape)

(3833, 10412)
(3833,)


In [39]:
model.fit(X_train, y_train, epochs=5, validation_data=(X_val, y_val))

Epoch 1/5
120/120 [==============================] - 89s 702ms/step - loss: 1.8459 - accuracy: 0.8552 - val_loss: 0.9388 - val_accuracy: 0.8926
Epoch 2/5
120/120 [==============================] - 80s 664ms/step - loss: 1.0218 - accuracy: 0.8722 - val_loss: 0.9913 - val_accuracy: 0.8926
Epoch 3/5
120/120 [==============================] - 80s 666ms/step - loss: 0.9183 - accuracy: 0.8722 - val_loss: 1.0820 - val_accuracy: 0.8926
Epoch 4/5
120/120 [==============================] - 79s 660ms/step - loss: 0.8891 - accuracy: 0.8722 - val_loss: 1.1052 - val_accuracy: 0.8926
Epoch 5/5
120/120 [==============================] - 80s 664ms/step - loss: 0.8840 - accuracy: 0.8722 - val_loss: 1.1169 - val_accuracy: 0.8926
